In [ ]:
import numpy as np
import tensorflow as tf
from keras import backend as K
import imageio
import pandas as pd
from keras.layers import Conv2D, Input, Lambda
from keras.models import load_model, Model
import PIL
import scipy.io
import scipy.misc
from skimage.transform import resize
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body
%matplotlib inline


Using TensorFlow backend.


ModuleNotFoundError: ignored

In [ ]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.6):
  box_scores = box_confidence * box_class_probs
  box_classes = K.argmax(box_scores, -1)
  box_class_scores = K.max(box_scores, -1)
  filtering_mask = box_class_scores > threshold
  scores = tf.boolean_mask(box_class_scores, filtering_mask)
  boxes = tf.boolean_mask(boxes, filtering_mask)
  classes = tf.boolean_mask(box_classes, filtering_mask)
  return scores, boxes, classes

In [ ]:
def iou(box1, box2):
  xi1 = max(box1[0], box2[0])
  yi1 = max(box1[1], box2[1])
  xi2 = max(box1[2], box2[2])
  yi2 = max(box1[3], box2[3])
  inter_area = (yi2-yi1)*(xi2-xi1)
  box1_area = (box[3]-box[1])*(box[2]-box[0])
  box2_area = (box[3]-box[1])*(box[2]-box[0])
  union_area = box1_area + box2_area - inter_area
  iou = inter_area/ union_area
  return iou

In [ ]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
  max_boxes_tensor = K.variable(max_boxes, dtype = 'int32')
  K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
  nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes, iou_threshold)
  scores = K.gather(scores, nms_indices)
  boxes = K.gather(boxes, nms_indices)
  classes = K.gather(classes, nms_indices)
  return scores, boxes, classes

In [ ]:
yolo_outputs = (tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed=1), tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed=1), tf.random_normal([19, 19, 5, 2], mean=1, stddev=2, seed=1), tf.random_normal([19, 19, 5, 80], mean=1, stddev=1, seed=1))

In [ ]:
def yolo_eval(yolo_output, image_shape = (720., 1280.), max_boxes = 10, score_threshold = 0.6, iou_threshold = 0.5):
  box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
  boxes = yolo_boxes_to_corners(box_xy, box_wh)
  scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = score_threshold)
  boxes = scale_boxes(boxes, image_shape)
  scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)
  return scores, boxes, classes

In [ ]:
scores, boxes, classes = yolo_eval(yolo_outputs)

In [ ]:
with tf.Session() as test_b:
  print('scores[2] = '+ str(scores[2].eval()))
  print('boxes[2] = '+ str(boxes[2].eval()))
  print('classes[2] = '+ str(classes[2].eval()))
  

In [ ]:
sess = K.get_session()
class_names = read_classes('drive/My Drive/model_data/coco_classes.txt')
anchors = read_anchors('drive/My Drive/model_data/yolo_anchors.txt')
yolo_model = load_model('drive/My Drive/model_data/yolo.h5')

In [ ]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

In [ ]:
def predict(sess, image_file):
    image, image_data = preprocess_image(image_file, model_image_size = (608, 608))
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], feed_dict={yolo_model.input: image_data, K.learning_phase(): 0})
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    colors = generate_colors(class_names)
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    image.save(os.path.join("out", image_file), quality=90)
    output_image = imageio.imread(os.path.join("out", image_file))
    plt.figure(figsize=(12,12))
    imshow(output_image)

    return out_scores, out_boxes, out_classes

In [ ]:
img = plt.imread('test.JPG')
image_shape = float(img.shape[0]), float(img.shape[1])
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)

In [ ]:
out_scores, out_boxes, out_classes = predict(sess, "testB.JPG")